*Tidy data*: é um *framework* para estruturar conjuntos de dados de modo que sejam facilmente analisados e visualizados.

**Objetivos**

Este capítulo abordará:
1. Operação de unpivoting/melting/gathering de colunas em linhas ("despivotear").
2. Operação de pivoting/casting/spreading de linhas em colunas ("pivotear").
3. Normalização de dados separando um dataframe em várias tabelas.
4. Reunião de dados de várias partes.

In [3]:
import numpy as np
import pandas as pd

# dados do Pew Research Center sobre renda e religião nos EUA
pew = pd.read_csv("pew.csv", encoding = "cp1252")
pew.head()

,religion,<$10k,$10-20k,$20-30k,$30-40k,$40-50k,$50-75k,$75-100k,$100-150k,>150k,Don't know/refused
0,Agnostic,27,34,60,81,76,137,122,109,84,96
1,Atheist,12,27,37,52,35,70,73,59,74,76
2,Buddhist,27,21,30,34,33,58,62,39,53,54
3,Catholic,418,617,732,670,638,1116,949,792,633,1489
4,Don’t know/refused,15,14,15,11,10,35,21,17,18,116


In [6]:
# há variáveis (e.g., renda) espalhados por várias colunas (wide format)
# isso precisa ser corrigido (reshape para long format)

# MELT, parâmetros:
# 1. id_vars: contêiner que representa as variáveis que permanecerão inalteradas;
# 2. value_vars: identifica as colunas em que a operação de melt (ou unpivot) será executada;
# 3. var_name: é uma string para o nome da nova coluna quando um melt é executado em value_vars;
# nome default: variable;
# 4. value_name: é uma string para o nome da nova coluna que representa os valores para var_name;
# nome default: value.

# não precisamos especificar um value_vars, pois queremos pivotear
# todas as colunas, execto a coluna "religion"
pew_long = pd.melt(pew, id_vars = "religion")
pew_long.head(5)

,religion,variable,value
0,Agnostic,<$10k,27
1,Atheist,<$10k,12
2,Buddhist,<$10k,27
3,Catholic,<$10k,418
4,Don’t know/refused,<$10k,15


In [7]:
pew_long.tail(5)

,religion,variable,value
175,Orthodox,Don't know/refused,73
176,Other Christian,Don't know/refused,18
177,Other Faiths,Don't know/refused,71
178,Other World Religions,Don't know/refused,8
179,Unaffiliated,Don't know/refused,597


In [8]:
# nomeando as colunas
pew_long = pd.melt(pew,
                 id_vars = "religion",
                 var_name = "income",
                 value_name = "count")

pew_long.head(5)

,religion,income,count
0,Agnostic,<$10k,27
1,Atheist,<$10k,12
2,Buddhist,<$10k,27
3,Catholic,<$10k,418
4,Don’t know/refused,<$10k,15


In [10]:
# mantendo várias colunas fixas
billboard = pd.read_csv("billboard.csv")
billboard.iloc[0:5, 0:16] # cada semana tem sua própria coluna

,year,artist,track,time,date.entered,wk1,wk2,wk3,wk4,wk5,wk6,wk7,wk8,wk9,wk10,wk11
0,2000,2 Pac,Baby Don't Cry (Keep...,4:22,2000-02-26,87,82.0,72.0,77.0,87.0,94.0,99.0,NaN,NaN,NaN,NaN
1,2000,2Ge+her,The Hardest Part Of ...,3:15,2000-09-02,91,87.0,92.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2000,3 Doors Down,Kryptonite,3:53,2000-04-08,81,70.0,68.0,67.0,66.0,57.0,54.0,53.0,51.0,51.0,51.0
3,2000,3 Doors Down,Loser,4:24,2000-10-21,76,76.0,72.0,69.0,67.0,65.0,55.0,59.0,62.0,61.0,61.0
4,2000,504 Boyz,Wobble Wobble,3:35,2000-04-15,57,34.0,25.0,17.0,17.0,31.0,36.0,49.0,53.0,57.0,64.0


In [12]:
# melt é necessário caso queiramos plotar semana como uma faceta em um gráfico
billboard_long = pd.melt(billboard,
                        id_vars = ["year", "artist", "track", "time", "date.entered"],
                        var_name = "week",
                        value_name = "rating")

billboard_long.head()

,year,artist,track,time,date.entered,week,rating
0,2000,2 Pac,Baby Don't Cry (Keep...,4:22,2000-02-26,wk1,87.0
1,2000,2Ge+her,The Hardest Part Of ...,3:15,2000-09-02,wk1,91.0
2,2000,3 Doors Down,Kryptonite,3:53,2000-04-08,wk1,81.0
3,2000,3 Doors Down,Loser,4:24,2000-10-21,wk1,76.0
4,2000,504 Boyz,Wobble Wobble,3:35,2000-04-15,wk1,57.0


In [13]:
billboard_long.tail()

,year,artist,track,time,date.entered,week,rating
24087,2000,Yankee Grey,Another Nine Minutes,3:10,2000-04-29,wk76,NaN
24088,2000,"Yearwood, Trisha",Real Live Woman,3:55,2000-04-01,wk76,NaN
24089,2000,Ying Yang Twins,Whistle While You Tw...,4:19,2000-03-18,wk76,NaN
24090,2000,Zombie Nation,Kernkraft 400,3:30,2000-09-02,wk76,NaN
24091,2000,matchbox twenty,Bent,4:12,2000-04-29,wk76,NaN


In [14]:
# colunas contendo diversas variávies
ebola = pd.read_csv("country_timeseries.csv")
ebola.columns

Index(['Date', 'Day', 'Cases_Guinea', 'Cases_Liberia', 'Cases_SierraLeone',
       'Cases_Nigeria', 'Cases_Senegal', 'Cases_UnitedStates', 'Cases_Spain',
       'Cases_Mali', 'Deaths_Guinea', 'Deaths_Liberia', 'Deaths_SierraLeone',
       'Deaths_Nigeria', 'Deaths_Senegal', 'Deaths_UnitedStates',
       'Deaths_Spain', 'Deaths_Mali'],
      dtype='object')

In [15]:
# exibe linhas selecionadas
# casos e mortes em duas colunas, mas Guinea e Liberia também estão em duas colunas
ebola.iloc[:5, [0, 1, 2, 3, 10, 11]]

,Date,Day,Cases_Guinea,Cases_Liberia,Deaths_Guinea,Deaths_Liberia
0,1/5/2015,289,2776.0,NaN,1786.0,NaN
1,1/4/2015,288,2775.0,NaN,1781.0,NaN
2,1/3/2015,287,2769.0,8166.0,1767.0,3496.0
3,1/2/2015,286,NaN,8157.0,NaN,3496.0
4,12/31/2014,284,2730.0,8115.0,1739.0,3471.0


In [16]:
ebola_long = pd.melt(ebola, id_vars = ["Date", "Day"])
ebola_long.head()

,Date,Day,variable,value
0,1/5/2015,289,Cases_Guinea,2776.0
1,1/4/2015,288,Cases_Guinea,2775.0
2,1/3/2015,287,Cases_Guinea,2769.0
3,1/2/2015,286,Cases_Guinea,NaN
4,12/31/2014,284,Cases_Guinea,2730.0


In [17]:
ebola_long.tail()

,Date,Day,variable,value
1947,3/27/2014,5,Deaths_Mali,NaN
1948,3/26/2014,4,Deaths_Mali,NaN
1949,3/25/2014,3,Deaths_Mali,NaN
1950,3/24/2014,2,Deaths_Mali,NaN
1951,3/22/2014,0,Deaths_Mali,NaN


In [19]:
# separar e adicionar colunas individualmente (método simples)
variable_split = ebola_long["variable"].str.split("_")
variable_split[:5]

0    [Cases, Guinea]
1    [Cases, Guinea]
2    [Cases, Guinea]
3    [Cases, Guinea]
4    [Cases, Guinea]
Name: variable, dtype: object

In [20]:
variable_split[:-5]

0       [Cases, Guinea]
1       [Cases, Guinea]
2       [Cases, Guinea]
3       [Cases, Guinea]
4       [Cases, Guinea]
             ...       
1942     [Deaths, Mali]
1943     [Deaths, Mali]
1944     [Deaths, Mali]
1945     [Deaths, Mali]
1946     [Deaths, Mali]
Name: variable, Length: 1947, dtype: object

In [21]:
status_values = variable_split.str.get(0)
status_values

0        Cases
1        Cases
2        Cases
3        Cases
4        Cases
         ...  
1947    Deaths
1948    Deaths
1949    Deaths
1950    Deaths
1951    Deaths
Name: variable, Length: 1952, dtype: object

In [22]:
country_values = variable_split.str.get(1)
country_values

0       Guinea
1       Guinea
2       Guinea
3       Guinea
4       Guinea
         ...  
1947      Mali
1948      Mali
1949      Mali
1950      Mali
1951      Mali
Name: variable, Length: 1952, dtype: object

In [23]:
ebola_long["status"] = status_values
ebola_long["country"] = country_values
ebola_long.head()

,Date,Day,variable,value,status,country
0,1/5/2015,289,Cases_Guinea,2776.0,Cases,Guinea
1,1/4/2015,288,Cases_Guinea,2775.0,Cases,Guinea
2,1/3/2015,287,Cases_Guinea,2769.0,Cases,Guinea
3,1/2/2015,286,Cases_Guinea,NaN,Cases,Guinea
4,12/31/2014,284,Cases_Guinea,2730.0,Cases,Guinea


In [24]:
# separar e combinar em um único passo (método simples)
variable_split = ebola_long["variable"].str.split("_", expand = True)
variable_split.columns = ["status", "country"]
ebola_parsed = pd.concat([ebola_long, variable_split], axis = 1)
ebola_parsed.head()

,Date,Day,variable,value,status,country,status,country
0,1/5/2015,289,Cases_Guinea,2776.0,Cases,Guinea,Cases,Guinea
1,1/4/2015,288,Cases_Guinea,2775.0,Cases,Guinea,Cases,Guinea
2,1/3/2015,287,Cases_Guinea,2769.0,Cases,Guinea,Cases,Guinea
3,1/2/2015,286,Cases_Guinea,NaN,Cases,Guinea,Cases,Guinea
4,12/31/2014,284,Cases_Guinea,2730.0,Cases,Guinea,Cases,Guinea


In [25]:
ebola_parsed.tail()

,Date,Day,variable,value,status,country,status,country
1947,3/27/2014,5,Deaths_Mali,NaN,Deaths,Mali,Deaths,Mali
1948,3/26/2014,4,Deaths_Mali,NaN,Deaths,Mali,Deaths,Mali
1949,3/25/2014,3,Deaths_Mali,NaN,Deaths,Mali,Deaths,Mali
1950,3/24/2014,2,Deaths_Mali,NaN,Deaths,Mali,Deaths,Mali
1951,3/22/2014,0,Deaths_Mali,NaN,Deaths,Mali,Deaths,Mali


In [27]:
# variáveis tanto em linhas quanto em colunas
weather = pd.read_csv("weather.csv")
weather.iloc[:5, :11]

,id,year,month,element,d1,d2,d3,d4,d5,d6,d7
0,MX17004,2010,1,tmax,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,MX17004,2010,1,tmin,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,MX17004,2010,2,tmax,NaN,27.3,24.1,NaN,NaN,NaN,NaN
3,MX17004,2010,2,tmin,NaN,14.4,14.4,NaN,NaN,NaN,NaN
4,MX17004,2010,3,tmax,NaN,NaN,NaN,NaN,32.1,NaN,NaN


In [29]:
weather_melt = pd.melt(weather,
                      id_vars = ["id", "year", "month", "element"],
                      var_name = "day",
                      value_name = "temp")
weather_melt.head()

,id,year,month,element,day,temp
0,MX17004,2010,1,tmax,d1,NaN
1,MX17004,2010,1,tmin,d1,NaN
2,MX17004,2010,2,tmax,d1,NaN
3,MX17004,2010,2,tmin,d1,NaN
4,MX17004,2010,3,tmax,d1,NaN


In [30]:
weather_melt.tail()

,id,year,month,element,day,temp
677,MX17004,2010,10,tmin,d31,NaN
678,MX17004,2010,11,tmax,d31,NaN
679,MX17004,2010,11,tmin,d31,NaN
680,MX17004,2010,12,tmax,d31,NaN
681,MX17004,2010,12,tmin,d31,NaN


In [31]:
# melt = função do Pandas
# pivot_table = método chamado em um objeto DataFrame
weather_tidy = weather_melt.pivot_table(index = ["id", "year", "month", "day"],
                                       columns = "element",
                                       values = "temp")

weather_tidy_flat = weather_tidy.reset_index()
weather_tidy_flat.head()

element,id,year,month,day,tmax,tmin
0,MX17004,2010,1,d30,27.8,14.5
1,MX17004,2010,2,d11,29.7,13.4
2,MX17004,2010,2,d2,27.3,14.4
3,MX17004,2010,2,d23,29.9,10.7
4,MX17004,2010,2,d3,24.1,14.4


In [32]:
# várias unidades de observação em uma tabela (normalização)
billboard_long.head()

,year,artist,track,time,date.entered,week,rating
0,2000,2 Pac,Baby Don't Cry (Keep...,4:22,2000-02-26,wk1,87.0
1,2000,2Ge+her,The Hardest Part Of ...,3:15,2000-09-02,wk1,91.0
2,2000,3 Doors Down,Kryptonite,3:53,2000-04-08,wk1,81.0
3,2000,3 Doors Down,Loser,4:24,2000-10-21,wk1,76.0
4,2000,504 Boyz,Wobble Wobble,3:35,2000-04-15,wk1,57.0


In [33]:
billboard_long[billboard_long["track"] == "Loser"].head()

,year,artist,track,time,date.entered,week,rating
3,2000,3 Doors Down,Loser,4:24,2000-10-21,wk1,76.0
320,2000,3 Doors Down,Loser,4:24,2000-10-21,wk2,76.0
637,2000,3 Doors Down,Loser,4:24,2000-10-21,wk3,72.0
954,2000,3 Doors Down,Loser,4:24,2000-10-21,wk4,69.0
1271,2000,3 Doors Down,Loser,4:24,2000-10-21,wk5,67.0


In [34]:
billboard_songs = billboard_long[["year", "artist", "track", "time"]]
billboard_songs.shape

(24092, 4)

In [35]:
billboard_songs = billboard_songs.drop_duplicates()
billboard_songs.shape

(317, 4)

In [36]:
billboard_songs["id"] = range(len(billboard_songs))
billboard_songs.head(n = 10)

,year,artist,track,time,id
0,2000,2 Pac,Baby Don't Cry (Keep...,4:22,0
1,2000,2Ge+her,The Hardest Part Of ...,3:15,1
2,2000,3 Doors Down,Kryptonite,3:53,2
3,2000,3 Doors Down,Loser,4:24,3
4,2000,504 Boyz,Wobble Wobble,3:35,4
5,2000,98^0,Give Me Just One Nig...,3:24,5
6,2000,A*Teens,Dancing Queen,3:44,6
7,2000,Aaliyah,I Don't Wanna,4:15,7
8,2000,Aaliyah,Try Again,4:03,8
9,2000,"Adams, Yolanda",Open My Heart,5:30,9


In [38]:
# combina o dataframe de músicas com o conjunto de dados original
billboard_ratings = billboard_long.merge(billboard_songs,
                                        on = ["year", "artist", "track", "time"])
billboard_ratings.shape

(24092, 8)